In [1]:
%pip install canvasapi

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv, find_dotenv
import os, json, re
import ast

load_dotenv()
    
data_dir = os.environ.get("DATA_DIR", "input")
loading_folder_env = os.environ.get("LOADING_FOLDER", '[""]')
try:
    loading_folder = json.loads(loading_folder_env)
except Exception:
    try:
        loading_folder = ast.literal_eval(loading_folder_env)
    except Exception:
        loading_folder = [s.strip() for s in re.split(r'[;,]', loading_folder_env) if s.strip()]
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
model_name = os.getenv("OPENAI_MODEL_NAME")

if not all([api_key, api_base, model_name]):
    raise RuntimeError("Check .env file and ensure that it exist variables OPENAI_API_KEY, OPENAI_API_BASE, and OPENAI_MODEL_NAME.")

API_URL = os.getenv("API_URL")
API_KEY = os.getenv("API_KEY")
PART_USED = os.getenv("PART_USED")
COURSE_ID = os.getenv("COURSE_ID")

In [3]:
print(model_name)
loading_folder

google/gemini-2.5-flash


['6.De so']

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
            model_name=model_name,
            openai_api_key=api_key,
            openai_api_base=api_base,
            max_tokens=None,
            max_retries=2,
            temperature=0.69
        )
def ask(request):
    try:
        #print(api_key)
        response = llm.invoke(request)
        return response
    except Exception as e:
            print(request)
            raise RuntimeError(f"Error when calling openAI: {e}")

c:\Users\QUANG TRUNG\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
prompt = None
candidates = [
    "prompt.txt"
]
seen = set()
candidates = [p for p in candidates if p and not (p in seen or seen.add(p))]

for p in candidates:
    if os.path.exists(p):
        with open(p, "r", encoding="utf-8") as _f:
            prompt = _f.read()
        print(f"Loaded prompt from: {p}")
        break
else:
    print("prompt.txt not found in candidate locations; prompt is None.")
print(prompt)
print(loading_folder)
for folder in loading_folder:
    pdf_files = []
    folder_path = os.path.join(data_dir, folder)
    for file in os.listdir(folder_path):
        if file.lower().endswith('.pdf'): #Duyet tung file pdf
            file = os.path.splitext(file)[0]
            pdf_files.append(file)
            file_path = os.path.abspath(os.path.join("..\\..\\output", folder, file))
            final_md_path = os.path.join(file_path, "output\\final.md")
            print(final_md_path)
            final_md_str = None
            if os.path.exists(final_md_path):
                with open(final_md_path, "r", encoding="utf-8") as f:
                    final_md_str = f.read()
            else:
                # fallback: try same basename as the PDF but with .md extension
                alt_md_path = os.path.splitext(file_path)[0] + ".md"
                if os.path.exists(alt_md_path):
                    with open(alt_md_path, "r", encoding="utf-8") as f:
                        final_md_str = f.read()
            try:
                response = ask(prompt + "\n\n" + final_md_str)
            except Exception as e:
                print("error prompting file", final_md_path)
                continue
            out_dir = os.path.dirname(final_md_path)
            os.makedirs(out_dir, exist_ok=True)
            base_name = os.path.splitext(os.path.basename(final_md_path))[0]
            out_path = os.path.join(out_dir, f"{base_name}.text2qti.txt")

            text = ""
            if response is None:
                text = ""
            elif isinstance(response, str):
                text = response
            else:
                if hasattr(response, "content"):
                    text = response.content
                elif hasattr(response, "text"):
                    text = response.text
                else:
                    choices = getattr(response, "choices", None)
                    if choices:
                        first = choices[0]
                        if isinstance(first, dict):
                            text = first.get("message", {}).get("content") or first.get("text", "") or ""
                        else:
                            # try common attributes on choice objects
                            text = getattr(first, "message", None) or getattr(first, "text", None) or str(first)
                    else:
                        # fallback to string representation
                        text = str(response)
            with open(out_path, "w", encoding="utf-8") as out_f:
                out_f.write(text or "")

            print(f"Wrote output to: {out_path}")
            print()

Loaded prompt from: prompt.txt
**VAI TRÒ:** Bạn là một script chuyển đổi dữ liệu tự động, không có khả năng sáng tạo. Nhiệm vụ duy nhất của bạn là phân tích văn bản Markdown chứa câu hỏi trắc nghiệm và chuyển đổi nó một cách chính xác tuyệt đối sang định dạng text2qti được chỉ định. Độ chính xác về định dạng là ưu tiên số một và duy nhất.

---

**QUY TẮC TUYỆT ĐỐI (PHẢI TUÂN THỦ 100%):**

1.  **QUY TẮC VÀNG - KHÔNG NGẮT DÒNG:**
    *   Toàn bộ nội dung của một **câu hỏi** (bao gồm văn bản, LaTeX, bảng `<table>`, hình ảnh `![](...)`) phải nằm trên **một dòng duy nhất**.
    *   Toàn bộ nội dung của một **lời giải** (bắt đầu bằng `...`) phải nằm trên **một dòng duy nhất**.
    *   **TUYỆT ĐỐI KHÔNG** chèn ký tự xuống dòng (`\n`) vào giữa nội dung câu hỏi hoặc lời giải dưới bất kỳ hình thức nào.

2.  **QUY TẮC DẤU CÁCH (QUAN TRỌNG):**
    *   Luôn phải có **một dấu cách (space) duy nhất** ngay sau các ký hiệu định danh đầu dòng.
    *   Áp dụng cho:
        *   Số thứ tự câu hỏi: `1. ` (V

KeyboardInterrupt: 

In [8]:
import subprocess, requests, time
from canvasapi import Canvas

# For each PDF folder scanned in previous cell, find the generated .text2qti.txt file and run text2qti on it.
for folder in loading_folder:
    folder_path = os.path.join(data_dir, folder)
    if not os.path.isdir(folder_path):
        print(f"Skipping missing folder: {folder_path}")
        continue

    for fname in os.listdir(folder_path):
        if not fname.lower().endswith(".pdf"):
            continue

        base = os.path.splitext(fname)[0]
        file_path = os.path.abspath(os.path.join(folder_path, base))
        # matches how the earlier cell built the final_md_path
        final_md_path = os.path.join(file_path, "output", "final.md")
        out_dir = os.path.dirname(final_md_path)
        base_name = os.path.splitext(os.path.basename(final_md_path))[0]  # "final"
        out_path = os.path.join(out_dir, f"{base_name}.text2qti.txt")

        if not os.path.exists(out_path):
            print(f"No text2qti output for: {out_path}")
            continue

        cmd = ["text2qti", out_path]
        print(f"Running: {' '.join(cmd)}")
        try:
            proc = subprocess.run(cmd, capture_output=True, text=True)
            if proc.returncode == 0:
                print(f"Success for {out_path}")
                if proc.stdout:
                    print("stdout:", proc.stdout.strip())
            else:
                print(f"Command failed (code={proc.returncode}) for {out_path}")
                if proc.stdout:
                    print("stdout:", proc.stdout.strip())
                if proc.stderr:
                    print("stderr:", proc.stderr.strip())

        except FileNotFoundError:
            print("Command `text2qti` not found. Ensure it's installed and on PATH.")
            break
        except Exception as e:
            print(f"Error running text2qti on {out_path}: {e}")
            continue

        def upload_qti_to_canvas(file_path: str):
            payload = {
                "migration_type": "qti_converter",
                "pre_attachment": {"name": "final.text2qti.zip"},
                "settings": {"import_quizzes_next": False}
            }
            headers = {
                "Authorization": f"Bearer {API_KEY}",
                "Content-Type": "application/json"
            }

            # Step 1: Create migration
            response = requests.post(
                f"{API_URL}/api/v1/courses/{COURSE_ID}/content_migrations/",
                headers=headers,
                data=json.dumps(payload)
            )
            response.raise_for_status()
            migration = response.json()
            upload_url = migration["pre_attachment"]["upload_url"]
            upload_params = migration["pre_attachment"]["upload_params"]

            # Step 2: Upload QTI zip
            with open(file_path, "rb") as file:
                files = {"file": (upload_params["Filename"], file)}
                upload_response = requests.post(upload_url, data=upload_params, files=files)
                upload_response.raise_for_status()

            # Step 3: Poll migration progress
            progress_url = migration["progress_url"]
            while True:
                progress_response = requests.get(progress_url, headers=headers)
                progress_response.raise_for_status()
                progress = progress_response.json()
                print(f"\rMigration progress: {progress['completion']}%", end="\n")
                if progress["workflow_state"] == "completed":
                    print("\nMigration completed successfully.")
                    break
                elif progress["workflow_state"] == "failed":
                    print("\nMigration failed.")
                    return None
                time.sleep(3)

            # Step 4: Find quiz by name
            canvas = Canvas(API_URL, API_KEY)
            course = canvas.get_course(COURSE_ID)
            quizzes = course.get_quizzes()

            # Base name (without number)
            base_name = f"[{folder}]"
            matching_quizzes = [q for q in quizzes if q.title.startswith(base_name)]

            if matching_quizzes:
                # If already exists → find the max quiz number
                numbers = []
                for q in matching_quizzes:
                    parts = q.title.split()
                    try:
                        numbers.append(int(parts[-1]))  # last token is the number
                    except ValueError:
                        pass
                next_num = max(numbers) + 1 if numbers else 1
            else:
                next_num = 1

            new_quiz = max(quizzes, key=lambda q: q.id, default=None)

            if new_quiz:
                new_title = f"{base_name} {base}"
                new_quiz.edit(quiz={'title': new_title})
                print(f"Renamed quiz: {new_title}")

                new_quiz.edit(quiz={'published': True})
                print("Quiz has been published.")

                return f"{API_URL}/courses/{COURSE_ID}/quizzes/{new_quiz.id}"
            else:
                print("No new quiz was created.")
                return None
        zip_path = os.path.join(file_path, "output", "final.text2qti.zip")
        if os.path.exists(zip_path):
            try:
                result = upload_qti_to_canvas(zip_path)
                if result:
                    print("Uploaded QTI:", result)
                else:
                    print("Upload completed but no quiz URL returned.")
            except Exception as e:
                print(f"Upload failed for {zip_path}: {e}")
        else:
            print(f"QTI zip not found: {zip_path}")
        #upload_qti_to_canvas(os.path.join(file_path, "output", "final.text2qti.zip"))

Running: text2qti d:\Python\DataForge-automated\MathTHPT2025\6.De so\1. Đề luyện thêm hàng ngày-Sở Ninh Bình\output\final.text2qti.txt
Success for d:\Python\DataForge-automated\MathTHPT2025\6.De so\1. Đề luyện thêm hàng ngày-Sở Ninh Bình\output\final.text2qti.txt
Migration progress: 0.0%
Migration progress: 50.5%
Migration progress: 95.0%
Migration progress: 100.0%

Migration completed successfully.
Renamed quiz: [6.De so] 1. Đề luyện thêm hàng ngày-Sở Ninh Bình
Quiz has been published.
Uploaded QTI: https://lms.uet.vnu.edu.vn//courses/2722/quizzes/18851
Running: text2qti d:\Python\DataForge-automated\MathTHPT2025\6.De so\10. Đề luyện thêm hàng ngày-Sở Quảng Nam\output\final.text2qti.txt
Command failed (code=1) for d:\Python\DataForge-automated\MathTHPT2025\6.De so\10. Đề luyện thêm hàng ngày-Sở Quảng Nam\output\final.text2qti.txt
stderr: Traceback (most recent call last):
  File "C:\Users\QUANG TRUNG\miniconda3\Lib\site-packages\text2qti\quiz.py", line 725, in __init__
    parse_actio